In [1]:
import time
import argparse
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import torch
import numpy as np
import cv2
from skimage.measure import compare_ssim, compare_psnr

import matplotlib.pyplot as plt
import utils
import dataset

def str2bool(v):
    #print(v)
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Unsupported value encountered.')

if __name__ == "__main__":
    # ----------------------------------------
    #        Initialize the parameters
    # ----------------------------------------
    parser = argparse.ArgumentParser()
    #GPU parameters
    parser.add_argument('--no_gpu', default = False, help = 'True for CPU')
    # Saving, and loading parameters
    
    #models_TinyDerainGAN_raindrop
    parser.add_argument('--save_name', type = str, default = '../results/models_TinyDerainGAN_epochs_raindrop', help = 'save the generated with certain epoch')
    parser.add_argument('--load_gname', type = str, default = '../models/models_TinyDerainGAN_raindrop/KPN_rainy_image_epoch40_bs4_generator.pth', help = 'load the pre-trained model with certain epoch')
    parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/raindrop/test_b', help = 'images baseroot')
    
    #TinyDerainGAN_RID
    #parser.add_argument('--save_name', type = str, default = '../results/TinyDerainGAN_epoch_test', help = 'save the generated with certain epoch')
    #arser.add_argument('--load_gname', type = str, default = '../models/models_TinyDerainGAN_raindrop/KPN_rainy_image_epoch200_bs4_generator.pth', help = 'load the pre-trained model with certain epoch')
    
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/RID/images', help = 'images baseroot')
    
    #models_SAUnet_DF_raindrop
    #parser.add_argument('--save_name', type = str, default = '../results/models_SAUnet_DF_raindrop_b', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = '../models/models_SAUnet_DF_raindrop/KPN_rainy_image_epoch200_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/raindrop/test_b', help = 'images baseroot')
    
    #models_SAUnet_DF_rain100H
    #parser.add_argument('--save_name', type = str, default = '../results/models_SAUnet_DF_rain100H', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = '../models/models_SAUnet_DF_rain100H/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/rain100H/test', help = 'images baseroot')
    
    #models_Original_KPN
    #parser.add_argument('--save_name', type = str, default = './results/models_Original_KPN', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_Original_KPN/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #models_original_KPN_rainmix_rain100H
    #parser.add_argument('--save_name', type = str, default = './results/models_original_KPN_rainmix_rain100H', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_original_KPN_rainmix_rain100H/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = './rainy_image_dataset/rain100H/test/', help = 'images baseroot')
    
    #models_original_KPN_Rain1400
    #parser.add_argument('--save_name', type = str, default = './results/models_original_KPN_Rain1400', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_original_KPN_Rain1400/KPN_rainy_image_epoch100_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/Rain1400/test/', help = 'images baseroot')
    
    #models_SAUnet_SASkipnet_rain100H
    #parser.add_argument('--save_name', type = str, default = './results/models_SAUnet_SASkipnet_rain100H', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SAUnet_SASkipnet_rain100H/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/rain100H/test/', help = 'images baseroot')
    
    #models_SAUnet_SASkipnet_notanh_Rain1400
    #parser.add_argument('--save_name', type = str, default = './results/models_SAUnet_SASkipnet_notanh_Rain1400', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SAUnet_SASkipnet_notanh_Rain1400/KPN_rainy_image_epoch100_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/Rain1400/test/', help = 'images baseroot')
   
    #parser.add_argument('--load_name', type = str, default = './models/KPN_single_image_epoch120_bs16_mu0_sigma30.pth', help = 'load the pre-trained model with certain epoch')
    parser.add_argument('--test_batch_size', type = int, default = 1, help = 'size of the batches')
    parser.add_argument('--num_workers', type = int, default = 1, help = 'number of workers')
    # Initialization parameters
    parser.add_argument('--init_type', type = str, default = 'xavier', help = 'initialization type of generator')
    parser.add_argument('--init_gain', type = float, default = 0.02, help = 'initialization gain of generator')
    # Dataset parameters
    parser.add_argument('--crop', type = str2bool, default = False, help = 'whether to crop input images')
    parser.add_argument('--crop_size', type = int, default = 512, help = 'single patch size')
    parser.add_argument('--resize', type = str2bool, default = True, help = 'whether to resize input images')
    parser.add_argument('--scale_size', type = int, default = 512, help = 'single patch size')
    parser.add_argument('--geometry_aug', type = str2bool, default = False, help = 'geometry augmentation (scaling)')
    parser.add_argument('--angle_aug', type = str2bool, default = False, help = 'geometry augmentation (rotation, flipping)')
    parser.add_argument('--scale_min', type = float, default = 1, help = 'min scaling factor')
    parser.add_argument('--scale_max', type = float, default = 1, help = 'max scaling factor')
    parser.add_argument('--add_noise', type = str2bool, default = False, help = 'whether to add noise to input images')
    parser.add_argument('--mu', type = int, default = 0, help = 'Gaussian noise mean')
    parser.add_argument('--sigma', type = int, default = 30, help = 'Gaussian noise variance: 30 | 50 | 70')
    opt = parser.parse_args(args=[])
    print(opt)

    # ----------------------------------------
    #                   Test
    # ----------------------------------------
    # Initialize
    if opt.no_gpu:
        generator = utils.create_generator(opt)
    else:
        generator = utils.create_generator(opt).cuda()

    test_dataset = dataset.DenoisingValDataset(opt)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = opt.test_batch_size, shuffle = False, num_workers = opt.num_workers, pin_memory = True)
    sample_folder = opt.save_name
    utils.check_path(sample_folder)

    psnr_sum, psnr_ave, ssim_sum, ssim_ave, time_sum, time_ave, eval_cnt, iter_psnr, iter_ssim = 0, 0, 0, 0, 0, 0, 0, 0,0
    #np.set_printoptions(threshold=np.inf)
    
    # Count start time
    prev_time = time.time()
    
    # forward
    for i, (true_input, true_target, height_origin, width_origin, image_original_name) in enumerate(test_loader):
        prev_time = time.time()

        # To device
        if opt.no_gpu:
            true_input = true_input
            true_target = true_target
        else:
            true_input = true_input.cuda()
            true_target = true_target.cuda()            

        # Forward propagation
        with torch.no_grad():
            #print(true_input.size()) 
            fake_target, feature_map = generator(true_input, true_input)
        
        #print(fake_target.shape, true_input.shape)
        
        time_iter = time.time() - prev_time
        time_sum = time_sum + time_iter
        #prev_time = time.time()

        # Save
        img_list = [true_input, fake_target, true_target]
        name_list = ['in', 'pred', 'gt']
        sample_name = '%d' % (i+1)
        #utils.save_sample_png(sample_folder = sample_folder, sample_name = '%d' % (i + 1), img_list = img_list, name_list = name_list, pixel_max_cnt = 255, height = height_origin, width = width_origin)
        
        # plot filter channel in grayscale
        
        #plt.imshow(feature_map[0][0].cpu().detach().numpy(), cmap='gray')
        #plt.savefig(sample_folder+'/'+ str(i+1) + '_featuremap.png')
        #fm_content=feature_map[0][0].cpu().detach().numpy()
        #print(fm_content)
        #utils.text_save(content = fm_content, filename = fm_file)
        
        # Evaluation
        #psnr_sum = psnr_sum + utils.psnr(cv2.imread(sample_folder + '/' + sample_name + '_' + name_list[1] + '.png').astype(np.float32), cv2.imread(sample_folder + '/' + sample_name + '_' + name_list[2] + '.png').astype(np.float32))
        img_pred_recover = utils.recover_process(fake_target, height = height_origin, width = width_origin)
        img_gt_recover = utils.recover_process(true_target, height = height_origin, width = width_origin)
        #psnr_sum = psnr_sum + utils.psnr(utils.recover_process(fake_target, height = height_origin, width = width_origin), utils.recover_process(true_target, height = height_origin, width = width_origin))
        iter_psnr = utils.psnr(img_pred_recover, img_gt_recover)
        iter_ssim = compare_ssim(img_gt_recover, img_pred_recover, multichannel = True, data_range = 255)
        psnr_sum = psnr_sum + iter_psnr
        ssim_sum = ssim_sum + iter_ssim
        
        eval_cnt = eval_cnt + 1
        print('The '+ str(i) +'-th iter: psnr: '+ str(round(iter_psnr,4)) +', ssim: '+ str(round(iter_ssim,4)) +', sec: ' + str(round(time_iter,6)))
        
    psnr_ave = psnr_sum / eval_cnt
    ssim_ave = ssim_sum / eval_cnt
    time_ave = time_sum / eval_cnt
    fps = 1/time_ave
    eval_file = "../data/evaluation_data.txt"
    
    psnr_content = opt.load_gname + ": psnr:" + str(round(psnr_ave,4)) + ", ssim:" + str(round(ssim_ave,4)) + ", fps: " + str(round(fps)) + "("+ str(round(time_ave,6)) +")\n"
    utils.text_save(content = psnr_content, filename = eval_file)
    
    
    


Namespace(add_noise=False, angle_aug=False, baseroot='../rainy_image_dataset/raindrop/test_b', crop=False, crop_size=512, geometry_aug=False, init_gain=0.02, init_type='xavier', load_gname='../models/models_TinyDerainGAN_raindrop/KPN_rainy_image_epoch40_bs4_generator.pth', mu=0, no_gpu=False, num_workers=1, resize=True, save_name='../results/models_TinyDerainGAN_epochs_raindrop', scale_max=1, scale_min=1, scale_size=512, sigma=30, test_batch_size=1)
Generator are loaded!
The 0-th iter: psnr: 23.5748, ssim: 0.802, sec: 0.055947
The 1-th iter: psnr: 24.3055, ssim: 0.8253, sec: 0.00893
The 2-th iter: psnr: 25.9698, ssim: 0.9097, sec: 0.010769
The 3-th iter: psnr: 29.1317, ssim: 0.8828, sec: 0.008698
The 4-th iter: psnr: 30.7414, ssim: 0.9375, sec: 0.00882
The 5-th iter: psnr: 18.0962, ssim: 0.892, sec: 0.008904
The 6-th iter: psnr: 23.4373, ssim: 0.7788, sec: 0.008678
The 7-th iter: psnr: 22.6195, ssim: 0.6873, sec: 0.009157
The 8-th iter: psnr: 24.9896, ssim: 0.9009, sec: 0.008942
The 9-

The 132-th iter: psnr: 24.7332, ssim: 0.8518, sec: 0.008438
The 133-th iter: psnr: 24.4319, ssim: 0.8251, sec: 0.008577
The 134-th iter: psnr: 24.1375, ssim: 0.8344, sec: 0.008357
The 135-th iter: psnr: 27.7551, ssim: 0.922, sec: 0.008301
The 136-th iter: psnr: 21.1714, ssim: 0.5664, sec: 0.008575
The 137-th iter: psnr: 21.9908, ssim: 0.7724, sec: 0.008327
The 138-th iter: psnr: 26.5657, ssim: 0.8577, sec: 0.008615
The 139-th iter: psnr: 26.0597, ssim: 0.9229, sec: 0.008288
The 140-th iter: psnr: 21.2349, ssim: 0.681, sec: 0.008657
The 141-th iter: psnr: 27.5247, ssim: 0.8794, sec: 0.008534
The 142-th iter: psnr: 23.9974, ssim: 0.8362, sec: 0.008548
The 143-th iter: psnr: 20.0878, ssim: 0.767, sec: 0.008269
The 144-th iter: psnr: 30.7821, ssim: 0.9425, sec: 0.008651
The 145-th iter: psnr: 28.8765, ssim: 0.87, sec: 0.008343
The 146-th iter: psnr: 27.9602, ssim: 0.924, sec: 0.008488
The 147-th iter: psnr: 26.5785, ssim: 0.9282, sec: 0.008573
The 148-th iter: psnr: 26.5924, ssim: 0.8927, 